In [1]:
"""DE Test Case."""
from pathlib import Path
import pandas as pd
import pomato

In [2]:
# wdir = Path("./examples/") # Change to local copy of examples folder
wdir = Path.cwd()
mato = pomato.POMATO(wdir=wdir, options_file="profiles/de.json")
mato.load_data('data_input/DE_2020.zip')

INFO - Creating Folder Structure
INFO - creating folder data_temp
INFO - creating folder data_temp/results_cache
INFO - creating folder data_temp/julia_files
INFO - creating folder data_temp/julia_files/data
INFO - creating folder data_temp/julia_files/results
INFO - creating folder data_temp/julia_files/cbco_data
INFO - Initializing DataObject
INFO - Initializing the GridTopology....
INFO - Initializing MarketModel...
INFO - Initializing the FBMCModule....
INFO - Initializing the GridTopology....
INFO - POMATO initialized!
INFO - Loading data from zipped data archive
WARNING - Not complete list of expected input data found. See .missing_data
INFO - Validating Input Data...
WARNING - Data validation completed with warnings. See the .data_validation_report.
WARNING - Some data was initialized empty. See model_validation_report.
WARNING - Some data missing or contained NaNs. See model_validation_report.
INFO - Calculating grid parameters!
INFO - The network consists of 25 components. Set

In [3]:
# Create the grid representation, the options are specified in the 
# json file supplied in the instantiation. 
# In this case the model runs a zonal pricing with NTC to neighboring countries
# and subsequent redispatch.
mato.create_grid_representation()
mato.run_market_model(update_data=True)

INFO - Start-Time: 23:22:51
INFO - Loading Clp...
INFO - Initialized
Starting Julia Process... |INFO - reading from file c:\users\riw\documents\repositories\pomato\pomato/daemon_market_model.json
INFO - Done with Initialization. Starting daemon process market_model!
INFO - Process ready!
Processing... |INFO - Using Clp Solver...
INFO - Starting with market_model
INFO - Run market model, including redispatch
INFO - Read Model Data..
INFO - Reading Model Data from: c:\Users\riw\Documents\repositories\pomato\examples/data_temp/julia_files/data/
INFO - Data Prepared
Processing... /INFO - Creating Results Folder
INFO - Splitting the model horizon in 1 segements.
INFO - Set storage regime in simplified model for 397 storage plants.
INFO - Solving Storage Model for 33 timesteps
INFO - Coin0506I Presolve 13143 (-423) rows, 30256 (-29045) columns and 57839 (-44134) elements
INFO - Clp0006I 0  Obj 97364857 Primal inf 5823102.5 (1813)
INFO - Clp0006I 290  Obj 97364857 Primal inf 6502199.2 (2022)


In [4]:
# Return the results, in this case two: 
# market result (uniform pricing, no network constraints)
# and redispatch (to account for N-0 network feasibility)
market_result, redisp_result = mato.data.return_results()

In [5]:
# Check for overloaded lines in the market and redispatch results
n0_m, _ = market_result.overloaded_lines_n_0()
print("Number of N-0 Overloads in market result: ", len(n0_m))
n0_r, _  = redisp_result.overloaded_lines_n_0()
print("Number of N-0 Overloads after redispatch: ", len(n0_r))

Number of N-0 Overloads in market result:  240
Number of N-0 Overloads after redispatch:  25


In [6]:
# Generation comparison between Market Result and Redispatch.
# Redispatch is calculated G_redispatch - G_market as delta
# The absolute delta represents the total redispatched energy
gen = pd.merge(
    market_result.data.plants[["plant_type", "fuel", "technology", "g_max", "node"]],
    market_result.G, left_index=True, right_on="p"
)
gen = pd.merge(gen, redisp_result.G, on=["p", "t"], suffixes=("_market", "_redispatch"))
gen["delta"] = gen["G_redispatch"] - gen["G_market"]
gen["delta_abs"] = gen["delta"].abs()
print("Redispatched energy per hour (abs) [MWh] ", gen.delta_abs.sum()/len(gen.t.unique()))

# Many of these functions are implemented as methods of the Result class
gen_2 = redisp_result.redispatch()
print("Redispatched energy per hour (abs) [MWh] ", gen_2.delta_abs.sum()/len(gen_2.t.unique()))


Redispatched energy per hour (abs) [MWh]  1157.6663542644126
Redispatched energy per hour (abs) [MWh]  1157.6663542644128


In [7]:
# Create Geo Plot. DISCLAiMER: The reported prices are the dual 
# in the redispatch result, thus including costs for redispatch.
mato.visualization.create_geo_plot(redisp_result, show_prices=True, show_plot=False, show_redispatch=True, filepath=mato.wdir.joinpath("redispatch.html"))


INFO - Estimated size in RAM for A is: 21.476448 MB
INFO - Done Calculating N-1 Flows


In [8]:
# Create visualization of the generation schedule in the market result. 
mato.visualization.create_generation_plot(market_result, show_plot=False, filepath=mato.wdir.joinpath("market_results.html"))



In [9]:
# 
mato._join_julia_instances()


INFO - EXIT
INFO - I 176590  Obj 3.4825539e+08 Primal inf 44567889 (12245)
INFO - Clp0006I 177388  Obj 3.4828481e+08 Primal inf 56482852 (11650)
INFO - Clp0006I 178186  Obj 3.4831243e+08 Primal inf 56244296 (10974)
INFO - Clp0006I 178984  Obj 3.4832211e+08 Primal inf 56171455 (10501)
INFO - Clp0006I 179782  Obj 3.4836395e+08 Primal inf 55770196 (9759)
INFO - Clp0006I 180580  Obj 3.4838945e+08 Primal inf 4.3837084e+08 (10577)
INFO - Clp0006I 181378  Obj 3.4842158e+08 Primal inf 1.0878034e+08 (9723)
INFO - Clp0006I 182176  Obj 3.4847614e+08 Primal inf 1.1943224e+08 (9288)
INFO - Clp0006I 182974  Obj 3.4850652e+08 Primal inf 1.2133736e+08 (9144)
INFO - Clp0006I 183176  Obj 3.4851195e+08 Primal inf 5.8035724e+09 (9493)
INFO - Clp0006I 183974  Obj 3.4854188e+08 Primal inf 1.6611451e+08 (9743)
INFO - Clp0006I 184772  Obj 3.4858763e+08 Primal inf 1.0158053e+08 (8869)
INFO - Clp0006I 185570  Obj 3.4862228e+08 Primal inf 46991581 (6993)
INFO - Clp0006I 186368  Obj 3.486638e+08 Primal inf 1.6129